In [1]:
# add ../oepul_chat to path 
from llama_index.evaluation import RetrieverEvaluator
import sys
sys.path.append('../oepul_chat')

In [2]:
from oepul_chat import load_index, get_header_path_mappings

In [5]:
index = load_index('indices/oepul_index/')
retriever = index.as_retriever(top_k=8)
documents = index.docstore.docs

header_path_mappings = get_header_path_mappings(documents)

# each element in the list as line in the file
with open('tmp/header_path_mappings.txt', 'w') as f:
    for key in header_path_mappings:
        f.write(f'{key}\n')

In [22]:
def get_ids_of_header_path(header_path, header_path_mappings):
    return header_path_mappings[header_path]


get_ids_of_header_path(
    'Vorbeugender Grundwasserschutz – Acker/FÖRDERBEDINGUNGEN/SCHLAGBEZOGENE AUFZEICHNUNGEN',
    header_path_mappings)

['6193a3bc-df78-4480-b945-21c523890867']

In [27]:
def get_doc_ids_by_query(query):
    res = retriever.retrieve(query)
    return [r.id_ for r in res]

get_doc_ids_by_query("Wie groß müssen die Schläge in der Maßnahme 'Vorbeugender Grundwasserschutz sein?")

['577cb7ff-6fd1-425a-bdcd-099455def520',
 '02a0e1b7-6238-4ea7-888e-11815d6e7a49']

In [30]:
eval = [{
    "query": "Wie groß müssen die Schläge in der Maßnahme 'Vorbeugender Grundwasserschutz sein?",
    "header_paths": [
        'Vorbeugender Grundwasserschutz – Acker/FÖRDERBEDINGUNGEN/SCHLAGBEZOGENE AUFZEICHNUNGEN'],
    "expected_answer": "Schläge müssen mindestens 0,3 ha groß sein."
}]

for item in eval: 
    query = item['query']
    expected_header_paths = item['header_paths']
    expected_answer = item['expected_answer']
    
    true_doc_ids = []
    for header_path in expected_header_paths:
        true_doc_ids.extend(get_ids_of_header_path(header_path, mappings))

    
    
    retriever_evaluator.evaluate(
        query=query, expected_ids=true_doc_ids
    )
    

NameError: name 'retriever_evaluator' is not defined